In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoImageProcessor, AutoModelForImageClassification
import cv2
import face_recognition
from PIL import Image
import os
import glob
from datetime import datetime
from tqdm import tqdm
import numpy as np
import pandas as pd

In [2]:
# ==============================================================================
# 1. CONFIGURATION
# ==============================================================================
ANALYSIS_OUTPUT_ROOT = "/Users/natalyagrokh/AI/ml_expressions/img_expressions/flywheel"
MODEL_PATH = "/Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V29_20250710_082807"
CENTROIDS_PATH = "/Users/natalyagrokh/AI/ml_expressions/img_expressions/flywheel/emotion_centroids.pt"
RELEVANCE_THRESHOLD = 0.6 
os.makedirs(ANALYSIS_OUTPUT_ROOT, exist_ok=True)

In [3]:
# ==============================================================================
# 2. HELPER FUNCTIONS
# ==============================================================================

def get_next_version(base_dir):
    all_entries = glob.glob(os.path.join(base_dir, "V*_*"))
    existing = [os.path.basename(d) for d in all_entries if os.path.isdir(d)]
    versions = [int(d[1:].split("_")[0]) for d in existing if d.startswith("V") and "_" in d and d[1:].split("_")[0].isdigit()]
    next_version = max(versions, default=0) + 1
    return f"V{next_version}"

def get_emotion_predictions(face_image, model, processor, device):
    """Runs the model and returns a structured dictionary including all probabilities."""
    inputs = processor(images=face_image, return_tensors="pt").to(device)
    with torch.no_grad():
        logits = model(**inputs).logits
    probabilities = F.softmax(logits, dim=1).squeeze()
    top_confidence, top_pred_idx = torch.max(probabilities, dim=0)
    top_pred_label = model.config.id2label[top_pred_idx.item()]
    entropy = -torch.sum(probabilities * torch.log(probabilities + 1e-9)).item()
    
    results = {
        "predicted_label": top_pred_label,
        "confidence": top_confidence.item(),
        "entropy": entropy
    }
    # This loop ensures all individual probabilities are added to the log
    for i, prob in enumerate(probabilities):
        label = model.config.id2label[i]
        results[f"prob_{label}"] = prob.item()
    return results

# Processes a video with a two-stage approach, ensuring all file paths
    # are correctly created and logged.
def analyze_video_faces(video_path, save_dir, model, embedding_model, processor, device, centroids, relevance_threshold, process_every_n_frames=1):
    if not os.path.exists(video_path):
        print(f"❌ Error: Video file not found at {video_path}")
        return []

    video_capture = cv2.VideoCapture(video_path)
    total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = video_capture.get(cv2.CAP_PROP_FPS)
    print(f"✅ Opened video: {os.path.basename(video_path)} ({total_frames} frames at {fps:.2f} fps)")

    all_results_log = []
    
    # Create the directory for all face crops at the start
    face_crop_dir = os.path.join(save_dir, "face_crops")
    os.makedirs(face_crop_dir, exist_ok=True)

    pbar = tqdm(total=total_frames, desc="Processing video frames")

    for frame_count in range(total_frames):
        ret, frame = video_capture.read()
        if not ret:
            break
            
        if frame_count % process_every_n_frames == 0:
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            face_locations = face_recognition.face_locations(rgb_frame)
            
            if face_locations:
                for i, (top, right, bottom, left) in enumerate(face_locations):
                    face_image_pil = Image.fromarray(rgb_frame[top:bottom, left:right])
                    inputs = processor(images=face_image_pil, return_tensors="pt").to(device)

                    with torch.no_grad():
                        embedding = embedding_model(**inputs).logits.squeeze()
                    
                    distances = {
                        label_id: torch.nn.functional.cosine_similarity(embedding, centroid, dim=0).item()
                        for label_id, centroid in centroids.items()
                    }
                    max_similarity = max(distances.values())
                    
                    if max_similarity >= relevance_threshold:
                        with torch.no_grad():
                            logits = model(**inputs).logits
                        
                        emotion_results = get_emotion_predictions(face_image_pil, model, processor, device)
                        
                        # THIS IS THE FIX:
                        # 1. Create the full, absolute path for the cropped face.
                        face_filename = os.path.join(face_crop_dir, f"frame_{frame_count}_face_{i}.png")
                        
                        # 2. Save the image to that path.
                        face_image_pil.save(face_filename)

                        # 3. Log this exact, verified path to the results.
                        log_entry = {
                            "timestamp_seconds": frame_count / fps,
                            "frame_number": frame_count,
                            "face_index": i,
                            "face_crop_path": face_filename, # <-- Log the correct, full path
                            "is_relevant": True,
                            "max_similarity": max_similarity,
                            **emotion_results
                        }
                        all_results_log.append(log_entry)
        pbar.update(1)
        
    pbar.close()
    video_capture.release()
    
    print(f"✅ Video processing complete. Logged {len(all_results_log)} relevant emotional events.")
    return all_results_log

In [4]:
# ==============================================================================
# 3. MAIN EXECUTION BLOCK
# ==============================================================================

# --- Setup ---
VERSION = get_next_version(ANALYSIS_OUTPUT_ROOT)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
VERSION_TAG = f"{VERSION}_{timestamp}"
SAVE_DIR = os.path.join(ANALYSIS_OUTPUT_ROOT, VERSION_TAG)
os.makedirs(SAVE_DIR, exist_ok=True)
print(f"📁 Created analysis output directory: {SAVE_DIR}")

# --- Load Assets ---
print(f"\n--- Loading assets ---")
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
classification_model = AutoModelForImageClassification.from_pretrained(MODEL_PATH).to(device).eval()
embedding_model = AutoModelForImageClassification.from_pretrained(MODEL_PATH)
embedding_model.classifier = nn.Identity()
embedding_model.to(device).eval()
processor = AutoImageProcessor.from_pretrained(MODEL_PATH)
emotion_centroids = torch.load(CENTROIDS_PATH, map_location=device)
print(f"✅ Models, processor, and centroids loaded onto {device}.")

# --- Run Video Analysis ---
video_to_process = "/Users/natalyagrokh/AI/ml_expressions/img_expressions/flywheel/sample_vids/StreetQs.mp4" 
analysis_log = analyze_video_faces(
    video_path=video_to_process, 
    save_dir=SAVE_DIR,
    model=classification_model,
    embedding_model=embedding_model,
    processor=processor,
    device=device,
    centroids=emotion_centroids,
    relevance_threshold=RELEVANCE_THRESHOLD,
    process_every_n_frames=15 # Using a larger step to speed up this diagnostic run
)

# --- Save and Analyze Results ---
if analysis_log:
    log_df = pd.DataFrame(analysis_log)
    csv_path = os.path.join(SAVE_DIR, "filtered_emotion_log.csv")
    log_df.to_csv(csv_path, index=False)
    print(f"\n✅ Successfully saved full analysis to: {csv_path}")

    # --- Perform Diagnostic Analysis on the NEW data ---
    print("\n--- DATA DIAGNOSTIC REPORT ---")
    
    # 1. Analyze 'confidence'
    print("\nConfidence Score Distribution:")
    print(log_df['confidence'].describe())
    
    # 2. Analyze 'entropy'
    print("\nEntropy Score Distribution:")
    print(log_df['entropy'].describe())
    
    # 3. Analyze the maximum probability for each prediction
    prob_columns = [col for col in log_df.columns if col.startswith('prob_')]
    if prob_columns:
        log_df['max_probability'] = log_df[prob_columns].max(axis=1)
        print("\nMax Emotion Probability Distribution:")
        print(log_df['max_probability'].describe())
    else:
        print("\nError: No 'prob_' columns found in the log DataFrame.")
    
    print("\n--- END OF REPORT ---")
    print("\nUse the statistical distribution values above to set informed thresholds for the next filtering step.")

else:
    print("\n⚠️ No relevant emotional events were detected.")

📁 Created analysis output directory: /Users/natalyagrokh/AI/ml_expressions/img_expressions/flywheel/V4_20250714_122050

--- Loading assets ---
✅ Models, processor, and centroids loaded onto mps.
✅ Opened video: StreetQs.mp4 (5657 frames at 30.00 fps)


Processing video frames: 100%|██████████████| 5657/5657 [01:54<00:00, 49.46it/s]


✅ Video processing complete. Logged 354 relevant emotional events.

✅ Successfully saved full analysis to: /Users/natalyagrokh/AI/ml_expressions/img_expressions/flywheel/V4_20250714_122050/filtered_emotion_log.csv

--- DATA DIAGNOSTIC REPORT ---

Confidence Score Distribution:
count    354.000000
mean       0.895038
std        0.078503
min        0.484652
25%        0.881003
50%        0.925737
75%        0.944186
max        0.983957
Name: confidence, dtype: float64

Entropy Score Distribution:
count    354.000000
mean       0.513642
std        0.258741
min        0.116967
25%        0.337323
50%        0.411385
75%        0.558090
max        1.622529
Name: entropy, dtype: float64

Max Emotion Probability Distribution:
count    354.000000
mean       0.895038
std        0.078503
min        0.484652
25%        0.881003
50%        0.925737
75%        0.944186
max        0.983957
Name: max_probability, dtype: float64

--- END OF REPORT ---

Use the statistical distribution values above to 